In [1]:
# load data
import pandas as pd
df = pd.read_csv("./Data/SortedPOIs.csv")
TOTAL = len(df)+1
COL = df.columns
print(TOTAL) # 1157571
# orders = df.iloc[:,2]
# #print(orders)
# dfh = pd.read_csv("./Data/hilbert.csv")
# print(len(dfh))
# df.iloc[:,3]=dfh
print(df)

1157571
               x          y    order  hilbert  prediction
0       -85.0511  -96.00330        0      NaN         NaN
1       -85.0511  -95.97350        1      NaN         NaN
2       -85.0494  -80.88040        2      NaN         NaN
3       -84.9574   88.66650        3      NaN         NaN
4       -84.9463   88.72900        4      NaN         NaN
5       -84.9463   88.72900        5      NaN         NaN
6       -84.9463   88.72900        6      NaN         NaN
7       -84.9463   88.72900        7      NaN         NaN
8       -84.8686   88.58830        8      NaN         NaN
9       -83.8626 -160.31200        9      NaN         NaN
10      -83.7507   85.75520       10      NaN         NaN
11      -83.0072   88.59010       11      NaN         NaN
12      -83.0034   97.15430       12      NaN         NaN
13      -82.2142   11.25000       13      NaN         NaN
14      -79.1350   88.65930       14      NaN         NaN
15      -79.1294   88.64200       15      NaN         NaN
16    

In [2]:
# df = df.sort_values(['hilbert'])
# # print(df)
# df = df.reset_index(drop=True)
# df.iloc[:,2] = orders
# print(df)

# import numpy as np
# print(np.any(np.isnan(df))) # should be false
# print(np.all(np.isfinite(df))) # should be true

In [3]:
# http://scikit-learn.org/stable/developers/performance.html
from sklearn import linear_model
from sklearn import ensemble
import ipdb
import time

stages = [1,10,100]
index = []
data = []

start_time=time.time()

for i in range(len(stages)):
    temp = []
    for j in range(stages[i]):
        df_ = pd.DataFrame(columns=COL)
        temp.append(df_)
    data.append(temp)
    
data[0][0] = df
nodatacount = 0

for i in range(len(stages)):
    temp = []
    print("======stage:"+str(i)+"======")
    
    for j in range(stages[i]):
        print("model:"+str(j))
        if len(data[i][j]) == 0:
            nodatacount += 1
            lr = index[0][0]
        else: 
            lr = ensemble.GradientBoostingRegressor()
#             lr = linear_model.LinearRegression()
            train_x = data[i][j].iloc[:,0:2]
            train_y = data[i][j].iloc[:,2]
            lr.fit(train_x,train_y)
        temp.append(lr)
        # allocate data
        if i < len(stages)-1:
            predicted = lr.predict(train_x)
            for pi in range(len(predicted)): # do not change the prediction
                #print(item)
                if predicted[pi] < 0:
                    predicted[pi] = 0
                if predicted[pi] >= TOTAL:
                    predicted[pi] = TOTAL-1   
            ModelID = ((predicted/TOTAL)*stages[i+1]).astype(int)
            data[i][j].iloc[:,-1]=ModelID
            for k in range(stages[i+1]):
                data[i+1][k] = data[i+1][k].append(data[i][j].loc[data[i][j]['prediction'] == k])
#             too slow!
#             for k in range(len(ModelID)):
#                 data[i+1][ModelID[k]].loc[data[i+1][ModelID[k]].size/3]=data[i][j].iloc[k,:]
#                 if k % 1000 == 0:
#                     print(k)
    index.append(temp)

elapsed_time = time.time() - start_time
print(elapsed_time)
print(nodatacount)

======stage:0======
model:0
======stage:1======
model:0
model:1
model:2
model:3
model:4
model:5
model:6
model:7
model:8
model:9
======stage:2======
model:0
model:1
model:2
model:3
model:4
model:5
model:6
model:7
model:8
model:9
model:10
model:11
model:12
model:13
model:14
model:15
model:16
model:17
model:18
model:19
model:20
model:21
model:22
model:23
model:24
model:25
model:26
model:27
model:28
model:29
model:30
model:31
model:32
model:33
model:34
model:35
model:36
model:37
model:38
model:39
model:40
model:41
model:42
model:43
model:44
model:45
model:46
model:47
model:48
model:49
model:50
model:51
model:52
model:53
model:54
model:55
model:56
model:57
model:58
model:59
model:60
model:61
model:62
model:63
model:64
model:65
model:66
model:67
model:68
model:69
model:70
model:71
model:72
model:73
model:74
model:75
model:76
model:77
model:78
model:79
model:80
model:81
model:82
model:83
model:84
model:85
model:86
model:87
model:88
model:89
model:90
model:91
model:92
model:93
model:94
model:9

In [4]:
# # model persistent
# from sklearn.externals import joblib

# for i in range(len(index)):
#     for j in range(len(index[i])):
#         joblib.dump(index[i][j], 'index_'+str(i)+'_'+str(j)+'.pkl')

In [5]:
import numpy as np
# use the generated stage index for prediction
def predict_single_item_with_stage_model(model, data, total_size):
    tempy = 0
    modelIndex = 0
    for m in range(len(model)):
#         tempy = model[m][modelIndex].predict(data)
        tempy = model[m][modelIndex].predict(data.iloc[0,:].reshape(1,-1))
#         tempy = model[m][modelIndex].predict(data.iloc[0,])
#         tempy = model[m][modelIndex].predict(data.iloc[0,0:2])
        if tempy < 0:
            tempy = 0
        if tempy >= total_size:
            tempy = total_size-1
        if m < len(model)-1:
            modelIndex = int((tempy/total_size)*len(model[m+1]))
    return tempy

In [6]:
def range_query(lower_x, upper_x, lower_y, upper_y, model, df):
    df1 = pd.DataFrame(data=[[lower_x,lower_y,]], columns=["x","y"], index=[0])
    lower_predicted_position = int(predict_single_item_with_stage_model(index, df1, len(df))[0])
    
    discrepancy = 0
    scan = 0
    number_of_results = 0
    
    true_lower_position = 0
    true_upper_position = 0
    
    predicted_lower_position_x = df.iloc[lower_predicted_position,0]
    predicted_lower_position_y = df.iloc[lower_predicted_position,1]
    
    shift = 0
    # the predicted position is actually lower, search forward
    if (predicted_lower_position_x < lower_x) or (predicted_lower_position_x == lower_x and predicted_lower_position_y < lower_y):
        while (predicted_lower_position_x < lower_x) or (predicted_lower_position_x == lower_x and predicted_lower_position_y < lower_y):
            shift += 1
            scan += 1
            predicted_lower_position_x = df.iloc[lower_predicted_position+shift,0]
            predicted_lower_position_y = df.iloc[lower_predicted_position+shift,1]
        true_lower_position = lower_predicted_position+shift
    else: # search backward
        while (predicted_lower_position_x > lower_x) or (predicted_lower_position_x == lower_x and predicted_lower_position_y > lower_y):
            shift += 1
            predicted_lower_position_x = df.iloc[lower_predicted_position-shift,0]
            predicted_lower_position_y = df.iloc[lower_predicted_position-shift,1]
        true_lower_position = lower_predicted_position-shift+1
    
    discrepancy = shift
    shift = 0
    current_x = df.iloc[true_lower_position,0]
    current_y = df.iloc[true_lower_position,1]
    while (current_x < upper_x) or (current_x == upper_x and current_y <= upper_y):
        shift += 1
        scan += 1
        if current_x >= lower_x and current_x <= upper_x and current_y >= lower_y and current_y <= upper_y:
            number_of_results += 1
#             print(current_x,current_y)
        current_x = df.iloc[true_lower_position+shift,0]
        current_y = df.iloc[true_lower_position+shift,1]
    true_upper_position = true_lower_position+shift-1
    
    print("discrepancy: ",discrepancy)
    print("scan: ",scan)
    print("results: ",number_of_results)
    return discrepancy, scan, number_of_results
            

In [30]:
# df[(df['x'] > 38.7900) & (df['x'] < 39.0118)]
# df[(df['x'] == 38.7900) & (df['y'] >= -77.2123)]
# df[(df['x'] == 39.0118) & (df['y'] <= -76.8835)]

# df[(df['x'] > 22.1481) & (df['x'] < 22.5457)]
# df[(df['x'] == 22.1481) & (df['y'] >= 113.8436)]
# df[(df['x'] == 22.5457) & (df['y'] <= 114.4379)]

# df[(df['x'] > 22.2201) & (df['x'] < 22.2841)]
# df[(df['x'] == 22.2201) & (df['y'] >= 114.1278)]
# df[(df['x'] == 22.2841) & (df['y'] <= 114.2572)]

,x,y,order,hilbert,prediction


In [7]:
# import ipdb
# ipdb.set_trace()
range_query(38.7900,39.0118,-77.2123,-76.8835,index,df) # Washington DC
range_query(22.1481,22.5457,113.8436,114.4379,index,df) # Hong Kong
range_query(22.2201,22.2841,114.1278,114.2572,index,df) # Hong Kong island

discrepancy:  42
scan:  7957
results:  3650
discrepancy:  17
scan:  2809
results:  1507
discrepancy:  44
scan:  1012
results:  386


D:\anaconda\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  


(44, 1012, 386)

In [15]:
print(len(df[(df['x'] >= 38.7900) & (df['x'] <= 39.0118) & (df['y'] >= -77.2123) & (df['y'] <= -76.8835)]))
print(len(df[(df['x'] >= 22.1481) & (df['x'] <= 22.5457) & (df['y'] >= 113.8436) & (df['y'] <= 114.4379)]))
print(len(df[(df['x'] >= 22.2201) & (df['x'] <= 22.2841) & (df['y'] >= 114.1278) & (df['y'] <= 114.2572)]))
df[(df['x'] >= 22.2201) & (df['x'] <= 22.2841) & (df['y'] >= 114.1278) & (df['y'] <= 114.2572)]

3650
1507
386


,x,y,order,hilbert,prediction
511722,22.2276,114.218,511722,NaN,4
511750,22.2341,114.171,511750,NaN,4
511753,22.2346,114.171,511753,NaN,4
511754,22.2346,114.171,511754,NaN,4
511758,22.2351,114.172,511758,NaN,4
511759,22.2351,114.172,511759,NaN,4
511765,22.2360,114.185,511765,NaN,4
511773,22.2391,114.223,511773,NaN,4
511793,22.2433,114.162,511793,NaN,4
511794,22.2433,114.162,511794,NaN,4


In [11]:
import numpy as np
# use the generated stage index for prediction
def predict_with_stage_model(model, data):
    totalSize = len(data)+1
    predicted_y = []
    tempy = 0
    for i in range(len(data)):
        modelIndex = 0
        for m in range(len(model)):
            tempy = model[m][modelIndex].predict(data.iloc[i,:].reshape(1,-1))
#             tempy = model[m][modelIndex].predict(data.iloc[i,])
            if tempy < 0:
                tempy = 0
            if tempy >= totalSize:
                tempy = totalSize-1
            if m < len(model)-1:
                modelIndex = int((tempy/totalSize)*len(model[m+1]))
                
        predicted_y.append(tempy)
        
    return predicted_y

In [ ]:
import matplotlib.pyplot as plt
import time

train_x=df.iloc[:,0:2]
train_y=df.iloc[:,2]
# print(df.iloc[1:3,3])
# print(train_x.iloc[1:3,])

start_time=time.time()
predicted_y = predict_with_stage_model(index, train_x)
elapsed_time = time.time() - start_time
print(elapsed_time)

fig, ax = plt.subplots()
ax.scatter(train_y, predicted_y, edgecolors=(0, 0, 0))
ax.plot([train_y.min(), train_y.max()], [train_y.min(), train_y.max()], 'k--', lw=4)
ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')
plt.show()

In [ ]:
# estimate the result
from sklearn.metrics import explained_variance_score
train_y=df.iloc[:,2]
print(len(train_y))
print(len(predicted_y))

# 1- Var(Ytrue - Ypred)/Var(Ytrue)  best: 1  the lower the worse
result = explained_variance_score(train_y, predicted_y)
print(result)

In [ ]:
# calculate scan item amount
# index = 0
# scan = 0
# totalscan = 0
# for i in range(len(df)):
#     index = predicted_y[i]
#     scan = abs(index-i) # distance to exact
#     totalscan += scan

# print(totalscan/len(df))


In [ ]:
# space filling curve range query
# 用 C++ 改写